In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn import metrics

In [30]:
# Loading final dataset
df = pd.read_csv('C:/Users/cpedr/OneDrive - Hertie School/Semester 4/Machine Learning/Project/all_features_merged.csv')
print(df.head())
print(df.shape)

#Counting NA
nan_count = df['energy_price'].isna().sum()
print(nan_count)

df = df.dropna(subset=['energy_price'])
df = df[df['Country'] != 'Combined_DE_LU_DE_AT_LU']
df.shape

               Date Country  Solar  Wind Onshore  Wind Offshore  \
0  2014-12-31 23:00      GB    0.0       4546.00         3165.0   
1  2015-01-01 00:00      AT    0.0        117.25            NaN   
2  2015-01-01 00:00      BE    0.0        246.00          420.0   
3  2015-01-01 00:00      CH    0.0          4.00            NaN   
4  2015-01-01 00:00      CZ    0.0           NaN            NaN   

   water_reservoirs_and_hydro_storage  energy_price  biomass  gas  nuclear  
0                                 NaN           NaN      NaN  NaN      NaN  
1                                 NaN           NaN      NaN  NaN      NaN  
2                                 NaN           NaN      NaN  NaN      NaN  
3                                 NaN           NaN      NaN  NaN      NaN  
4                                 NaN           NaN      NaN  NaN      NaN  
(5551063, 10)
2179723


(3294780, 10)

In [132]:
df.dtypes

Date                                   object
Country                                object
Solar                                 float64
Wind Onshore                          float64
Wind Offshore                         float64
water_reservoirs_and_hydro_storage    float64
energy_price                          float64
biomass                               float64
gas                                   float64
nuclear                               float64
dtype: object

### Handling Dates

In [31]:
# Helper function to handle multiple date formats
def convert_dates(df, column_name, formats):
    # Start with a copy of the column to avoid altering the original data
    temp_series = pd.Series(pd.NaT, index=df.index)
    
    # Try each format and update only NaT entries
    for fmt in formats:
        temp_series = temp_series.combine_first(pd.to_datetime(df[column_name], format=fmt, errors='coerce'))
    
    return temp_series

# List of date formats you expect in your data
date_formats = ['%Y-%m-%d %H:%M:%S', '%Y-%m-%d %H:%M']

# Convert the 'dates' column to datetime
df['Date'] = convert_dates(df, 'Date', date_formats)

# Extract components
df['year'] = df['Date'].dt.year.astype('Int64')
df['month'] = df['Date'].dt.month.astype('Int64')
df['day'] = df['Date'].dt.day.astype('Int64')
df['hour'] = df['Date'].dt.hour.astype('Int64')
df['week_number'] = df['Date'].dt.isocalendar().week.astype('Int64')

# Extract the day of the week as an integer (Monday=0, Sunday=6)
df['day_of_week'] = df['Date'].dt.dayofweek

df

,Date,Country,Solar,Wind Onshore,Wind Offshore,water_reservoirs_and_hydro_storage,energy_price,biomass,gas,nuclear,year,month,day,hour,week_number,day_of_week
1976013,2015-01-01,CH,NaN,NaN,NaN,NaN,44.94,NaN,NaN,NaN,2015,1,1,0,1,3
1976014,2015-01-01,CZ,NaN,NaN,NaN,NaN,26.48,135.0,172.0,2596.0,2015,1,1,0,1,3
1976017,2015-01-01,DK_1,NaN,NaN,NaN,NaN,25.02,18.0,233.0,NaN,2015,1,1,0,1,3
1976018,2015-01-01,DK_2,NaN,NaN,NaN,NaN,27.38,25.0,304.0,NaN,2015,1,1,0,1,3
1976019,2015-01-01,EE,NaN,NaN,NaN,NaN,27.38,NaN,NaN,NaN,2015,1,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5551058,2023-12-31,SE_2,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,2023,12,31,0,52,6
5551059,2023-12-31,SE_3,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,2023,12,31,0,52,6
5551060,2023-12-31,SE_4,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,2023,12,31,0,52,6
5551061,2023-12-31,SI,NaN,NaN,NaN,NaN,33.30,NaN,NaN,NaN,2023,12,31,0,52,6


In [32]:
df.to_csv('C:/Users/cpedr/OneDrive - Hertie School/Semester 4/Machine Learning/Project/dates_features.csv')

# Only dummies (except the energy variables)

In [69]:
# Loading final dataset
df = pd.read_csv('C:/Users/cpedr/OneDrive - Hertie School/Semester 4/Machine Learning/Project/dates_features.csv')

df.drop('Unnamed: 0', axis=1, inplace=True)
df

,Date,Country,Solar,Wind Onshore,Wind Offshore,water_reservoirs_and_hydro_storage,energy_price,biomass,gas,nuclear,year,month,day,hour,week_number,day_of_week
0,2015-01-01 00:00:00,CH,NaN,NaN,NaN,NaN,44.94,NaN,NaN,NaN,2015,1,1,0,1,3
1,2015-01-01 00:00:00,CZ,NaN,NaN,NaN,NaN,26.48,135.0,172.0,2596.0,2015,1,1,0,1,3
2,2015-01-01 00:00:00,DK_1,NaN,NaN,NaN,NaN,25.02,18.0,233.0,NaN,2015,1,1,0,1,3
3,2015-01-01 00:00:00,DK_2,NaN,NaN,NaN,NaN,27.38,25.0,304.0,NaN,2015,1,1,0,1,3
4,2015-01-01 00:00:00,EE,NaN,NaN,NaN,NaN,27.38,NaN,NaN,NaN,2015,1,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294775,2023-12-31 00:00:00,SE_2,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,2023,12,31,0,52,6
3294776,2023-12-31 00:00:00,SE_3,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,2023,12,31,0,52,6
3294777,2023-12-31 00:00:00,SE_4,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,2023,12,31,0,52,6
3294778,2023-12-31 00:00:00,SI,NaN,NaN,NaN,NaN,33.30,NaN,NaN,NaN,2023,12,31,0,52,6


In [70]:
# Create a 'weekend' dummy variable where Saturday and Sunday are marked as 1, others as 0
df['weekend'] = (df['day_of_week'] >= 5).astype(int)

In [71]:
# List of columns for which to create dummies
columns_to_dummy = ['Country','year', 'month', 'day', 'hour', 'week_number', 'day_of_week']

# Loop through each column and create dummy variables with a prefix
for column in columns_to_dummy:
    dummies = pd.get_dummies(df[column], prefix=column).astype(int)
    df = pd.concat([df, dummies], axis=1)

df

,Date,Country,Solar,Wind Onshore,Wind Offshore,water_reservoirs_and_hydro_storage,energy_price,biomass,gas,nuclear,...,week_number_51,week_number_52,week_number_53,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,2015-01-01 00:00:00,CH,NaN,NaN,NaN,NaN,44.94,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
1,2015-01-01 00:00:00,CZ,NaN,NaN,NaN,NaN,26.48,135.0,172.0,2596.0,...,0,0,0,0,0,0,1,0,0,0
2,2015-01-01 00:00:00,DK_1,NaN,NaN,NaN,NaN,25.02,18.0,233.0,NaN,...,0,0,0,0,0,0,1,0,0,0
3,2015-01-01 00:00:00,DK_2,NaN,NaN,NaN,NaN,27.38,25.0,304.0,NaN,...,0,0,0,0,0,0,1,0,0,0
4,2015-01-01 00:00:00,EE,NaN,NaN,NaN,NaN,27.38,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294775,2023-12-31 00:00:00,SE_2,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,1
3294776,2023-12-31 00:00:00,SE_3,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,1
3294777,2023-12-31 00:00:00,SE_4,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,1
3294778,2023-12-31 00:00:00,SI,NaN,NaN,NaN,NaN,33.30,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,1


In [72]:
for column in df.columns :
    print(column)

Date
Country
Solar
Wind Onshore
Wind Offshore
water_reservoirs_and_hydro_storage
energy_price
biomass
gas
nuclear
year
month
day
hour
week_number
day_of_week
weekend
Country_BE
Country_BG
Country_CH
Country_CZ
Country_Combined_AT_DE_AT_LU
Country_DK_1
Country_DK_2
Country_EE
Country_ES
Country_FI
Country_FR
Country_GB
Country_GR
Country_HR
Country_HU
Country_IT_BRNN
Country_IT_CNOR
Country_IT_CSUD
Country_IT_FOGN
Country_IT_GR
Country_IT_NORD
Country_IT_NORD_AT
Country_IT_NORD_CH
Country_IT_NORD_FR
Country_IT_NORD_SI
Country_IT_PRGP
Country_IT_SACO_AC
Country_IT_SACO_DC
Country_IT_SARD
Country_IT_SICI
Country_IT_SUD
Country_LV
Country_ME
Country_MK
Country_NL
Country_NO_1
Country_NO_2
Country_NO_3
Country_NO_4
Country_NO_5
Country_PL
Country_PT
Country_RO
Country_RS
Country_SE_1
Country_SE_2
Country_SE_3
Country_SE_4
Country_SI
Country_SK
year_2015
year_2016
year_2017
year_2018
year_2019
year_2020
year_2021
year_2022
year_2023
month_1
month_2
month_3
month_4
month_5
month_6
month_7
mon

In [73]:
df.to_csv('C:/Users/cpedr/OneDrive - Hertie School/Semester 4/Machine Learning/Project/all_dummies.csv')

# Imputing Missing Values

In [42]:
df1 = pd.read_csv('C:/Users/cpedr/OneDrive - Hertie School/Semester 4/Machine Learning/Project/dates_features.csv')
df1.drop('Unnamed: 0', axis=1, inplace=True)
df1.isna().sum() 

Date                                        0
Country                                     0
Solar                                 3294780
Wind Onshore                          3294780
Wind Offshore                         3294780
water_reservoirs_and_hydro_storage    3294780
energy_price                                0
biomass                               1537435
gas                                   1237418
nuclear                               2514728
year                                        0
month                                       0
day                                         0
hour                                        0
week_number                                 0
day_of_week                                 0
dtype: int64

In [43]:
from sklearn.impute import SimpleImputer

In [44]:
df1.drop(['Solar', 'Wind Onshore', 'Wind Offshore', 'water_reservoirs_and_hydro_storage'], axis=1, inplace=True)

# Create an instance of SimpleImputer with mean strategy
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Apply fit_transform directly and replace in the DataFrame
columns_to_impute = ['biomass', 'gas', 'nuclear']
df1[columns_to_impute] = imputer.fit_transform(df1[columns_to_impute])

df1

,Date,Country,energy_price,biomass,gas,nuclear,year,month,day,hour,week_number,day_of_week
0,2015-01-01 00:00:00,CH,44.94,252.200902,1180.283774,7185.089448,2015,1,1,0,1,3
1,2015-01-01 00:00:00,CZ,26.48,135.000000,172.000000,2596.000000,2015,1,1,0,1,3
2,2015-01-01 00:00:00,DK_1,25.02,18.000000,233.000000,7185.089448,2015,1,1,0,1,3
3,2015-01-01 00:00:00,DK_2,27.38,25.000000,304.000000,7185.089448,2015,1,1,0,1,3
4,2015-01-01 00:00:00,EE,27.38,252.200902,1180.283774,7185.089448,2015,1,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
3294775,2023-12-31 00:00:00,SE_2,44.87,252.200902,1180.283774,7185.089448,2023,12,31,0,52,6
3294776,2023-12-31 00:00:00,SE_3,44.87,252.200902,1180.283774,7185.089448,2023,12,31,0,52,6
3294777,2023-12-31 00:00:00,SE_4,44.87,252.200902,1180.283774,7185.089448,2023,12,31,0,52,6
3294778,2023-12-31 00:00:00,SI,33.30,252.200902,1180.283774,7185.089448,2023,12,31,0,52,6


In [46]:
df1.isna().sum() 

Date            0
Country         0
energy_price    0
biomass         0
gas             0
nuclear         0
year            0
month           0
day             0
hour            0
week_number     0
day_of_week     0
dtype: int64

In [47]:
df1.to_csv('C:/Users/cpedr/OneDrive - Hertie School/Semester 4/Machine Learning/Project/dates_cleaned.csv')

In [74]:
df = pd.read_csv('C:/Users/cpedr/OneDrive - Hertie School/Semester 4/Machine Learning/Project/all_dummies.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df = df.dropna(subset=['energy_price'])
df

,Date,Country,Solar,Wind Onshore,Wind Offshore,water_reservoirs_and_hydro_storage,energy_price,biomass,gas,nuclear,...,week_number_51,week_number_52,week_number_53,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,2015-01-01 00:00:00,CH,NaN,NaN,NaN,NaN,44.94,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
1,2015-01-01 00:00:00,CZ,NaN,NaN,NaN,NaN,26.48,135.0,172.0,2596.0,...,0,0,0,0,0,0,1,0,0,0
2,2015-01-01 00:00:00,DK_1,NaN,NaN,NaN,NaN,25.02,18.0,233.0,NaN,...,0,0,0,0,0,0,1,0,0,0
3,2015-01-01 00:00:00,DK_2,NaN,NaN,NaN,NaN,27.38,25.0,304.0,NaN,...,0,0,0,0,0,0,1,0,0,0
4,2015-01-01 00:00:00,EE,NaN,NaN,NaN,NaN,27.38,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294775,2023-12-31 00:00:00,SE_2,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,1
3294776,2023-12-31 00:00:00,SE_3,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,1
3294777,2023-12-31 00:00:00,SE_4,NaN,NaN,NaN,NaN,44.87,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,1
3294778,2023-12-31 00:00:00,SI,NaN,NaN,NaN,NaN,33.30,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,1


In [75]:
df.drop(['Solar', 'Wind Onshore', 'Wind Offshore', 'water_reservoirs_and_hydro_storage'], axis=1, inplace=True)

# Create an instance of SimpleImputer with mean strategy
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Apply fit_transform directly and replace in the DataFrame
columns_to_impute = ['biomass', 'gas', 'nuclear']
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

df

,Date,Country,energy_price,biomass,gas,nuclear,year,month,day,hour,...,week_number_51,week_number_52,week_number_53,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,2015-01-01 00:00:00,CH,44.94,252.200902,1180.283774,7185.089448,2015,1,1,0,...,0,0,0,0,0,0,1,0,0,0
1,2015-01-01 00:00:00,CZ,26.48,135.000000,172.000000,2596.000000,2015,1,1,0,...,0,0,0,0,0,0,1,0,0,0
2,2015-01-01 00:00:00,DK_1,25.02,18.000000,233.000000,7185.089448,2015,1,1,0,...,0,0,0,0,0,0,1,0,0,0
3,2015-01-01 00:00:00,DK_2,27.38,25.000000,304.000000,7185.089448,2015,1,1,0,...,0,0,0,0,0,0,1,0,0,0
4,2015-01-01 00:00:00,EE,27.38,252.200902,1180.283774,7185.089448,2015,1,1,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294775,2023-12-31 00:00:00,SE_2,44.87,252.200902,1180.283774,7185.089448,2023,12,31,0,...,0,1,0,0,0,0,0,0,0,1
3294776,2023-12-31 00:00:00,SE_3,44.87,252.200902,1180.283774,7185.089448,2023,12,31,0,...,0,1,0,0,0,0,0,0,0,1
3294777,2023-12-31 00:00:00,SE_4,44.87,252.200902,1180.283774,7185.089448,2023,12,31,0,...,0,1,0,0,0,0,0,0,0,1
3294778,2023-12-31 00:00:00,SI,33.30,252.200902,1180.283774,7185.089448,2023,12,31,0,...,0,1,0,0,0,0,0,0,0,1


In [76]:
df.isna().sum() 

Date             0
Country          0
energy_price     0
biomass          0
gas              0
                ..
day_of_week_2    0
day_of_week_3    0
day_of_week_4    0
day_of_week_5    0
day_of_week_6    0
Length: 199, dtype: int64

In [77]:
df.to_csv('C:/Users/cpedr/OneDrive - Hertie School/Semester 4/Machine Learning/Project/dummies_cleaned.csv')

# XGBoost

### Cross Validation Split

In [7]:
df = pd.read_csv('C:/Users/cpedr/OneDrive - Hertie School/Semester 4/Machine Learning/Project/dummies_cleaned.csv')

In [8]:
from sklearn.model_selection import TimeSeriesSplit

# Shift
df['energy_price_target'] = df.groupby('Country')['energy_price'].shift(-12)
df.dropna(inplace=True) 

# Sort by date
df.sort_index(level='Date', inplace=True)
X = df.drop(['energy_price', 'energy_price_target', 'Country', 'Date'], axis=1)  # Exclude original price from features
y = df['energy_price_target']

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=20)

# Apply TimeSeriesSplit to df
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    print(f"Fold {i}:")
    print(f"  Train Indices: {train_index}")
    print(f"  Test Indices: {test_index}")


Fold 0:
  Train Indices: [     0      1      2 ... 156877 156878 156879]
  Test Indices: [156880 156881 156882 ... 313742 313743 313744]
Fold 1:
  Train Indices: [     0      1      2 ... 313742 313743 313744]
  Test Indices: [313745 313746 313747 ... 470607 470608 470609]
Fold 2:
  Train Indices: [     0      1      2 ... 470607 470608 470609]
  Test Indices: [470610 470611 470612 ... 627472 627473 627474]
Fold 3:
  Train Indices: [     0      1      2 ... 627472 627473 627474]
  Test Indices: [627475 627476 627477 ... 784337 784338 784339]
Fold 4:
  Train Indices: [     0      1      2 ... 784337 784338 784339]
  Test Indices: [784340 784341 784342 ... 941202 941203 941204]
Fold 5:
  Train Indices: [     0      1      2 ... 941202 941203 941204]
  Test Indices: [ 941205  941206  941207 ... 1098067 1098068 1098069]
Fold 6:
  Train Indices: [      0       1       2 ... 1098067 1098068 1098069]
  Test Indices: [1098070 1098071 1098072 ... 1254932 1254933 1254934]
Fold 7:
  Train Indices

In [13]:
from xgboost import XGBRegressor

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

In [10]:
# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
# Initialize and train the XGBoost classifier
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [17]:
param_grid = {
    'max_depth': [3, 4, 5, 6, 7],
    'min_child_weight': [1, 2, 3, 4],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150, 200],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9]
}
# Initialize GridSearchCV and fit to find the best parameters
grid_search = GridSearchCV(xgb, param_grid, scoring='neg_mean_squared_error', cv=tscv, verbose=1)
grid_search.fit(X_train_scaled, y_train)
# Print best parameters found by GridSearchCV
best_params = grid_search.best_params_
print("Best parameters:", best_params)

Fitting 20 folds for each of 5120 candidates, totalling 102400 fits


MemoryError: Unable to allocate 898. MiB for an array with shape (597600, 197) and data type float64